# 📊 TEMPO Project Status Dashboard
## Visual Overview of Data Files and Structure

This notebook provides a complete visual summary of:
- What data folders exist
- File sizes and row counts
- Column structures
- What's missing vs. what you have

In [ ]:
import pandas as pd
import os
from pathlib import Path
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

## 1. 📂 Directory Structure Overview

In [ ]:
# Expected data folders
expected_folders = [
    'baseline_data',
    'crisis_datasets', 
    'goemotion_data',
    'master_training_data',
    'non_crisis_data',
    'standardized_data'
]

print("=" * 80)
print("DATA FOLDER STATUS")
print("=" * 80)
print()

folder_status = {}
for folder in expected_folders:
    exists = Path(folder).exists()
    status = "✅ EXISTS" if exists else "❌ MISSING"
    folder_status[folder] = exists
    
    # Get size if exists
    if exists:
        total_size = sum(f.stat().st_size for f in Path(folder).rglob('*') if f.is_file())
        size_mb = total_size / (1024 * 1024)
        print(f"{status}  {folder:<30} ({size_mb:>8.1f} MB)")
    else:
        print(f"{status}  {folder:<30} (NOT DOWNLOADED)")

print()
print(f"Summary: {sum(folder_status.values())}/{len(expected_folders)} folders present")

## 2. 📄 CSV Files Inventory

In [ ]:
# Find all CSV files
csv_files = list(Path('.').rglob('*.csv'))

# Filter out git and venv folders
csv_files = [f for f in csv_files if '.git' not in str(f) and 'venv' not in str(f) and '.venv' not in str(f)]

print("=" * 80)
print(f"FOUND {len(csv_files)} CSV FILES")
print("=" * 80)
print()

# Group by folder
from collections import defaultdict
files_by_folder = defaultdict(list)

for f in csv_files:
    folder = str(f.parent)
    size_mb = f.stat().st_size / (1024 * 1024)
    files_by_folder[folder].append({
        'name': f.name,
        'size_mb': size_mb,
        'path': str(f)
    })

# Display by folder
for folder in sorted(files_by_folder.keys()):
    print(f"\n📁 {folder}/")
    print("-" * 80)
    for file_info in sorted(files_by_folder[folder], key=lambda x: x['name']):
        print(f"   {file_info['name']:<45} {file_info['size_mb']:>8.2f} MB")

## 3. 📊 Dataset Summary Table

In [ ]:
# Key datasets to analyze
key_datasets = [
    ('standardized_data/crisis_combined.csv', 'Crisis Combined'),
    ('standardized_data/non_crisis_combined.csv', 'Non-Crisis Combined'),
    ('standardized_data/humaid_standardized.csv', 'HumAID Standardized'),
    ('standardized_data/crisislex_standardized.csv', 'CrisisLex Standardized'),
    ('goemotion_data/goemotions.csv', 'GoEmotions (27 emotions)'),
    ('master_training_data/master_training_data.csv', 'Master Training File'),
]

dataset_summary = []

print("=" * 80)
print("KEY DATASETS ANALYSIS")
print("=" * 80)
print()

for filepath, name in key_datasets:
    if Path(filepath).exists():
        # Quick check using line count for very large files
        try:
            df = pd.read_csv(filepath, nrows=0)  # Just get columns
            columns = list(df.columns)
            
            # Count rows efficiently
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                row_count = sum(1 for line in f) - 1  # Subtract header
            
            size_mb = Path(filepath).stat().st_size / (1024 * 1024)
            
            dataset_summary.append({
                'Dataset': name,
                'Status': '✅',
                'Rows': f"{row_count:,}",
                'Columns': len(columns),
                'Size (MB)': f"{size_mb:.1f}"
            })
            
            print(f"✅ {name}")
            print(f"   Rows: {row_count:,} | Columns: {len(columns)} | Size: {size_mb:.1f} MB")
            print(f"   Columns: {', '.join(columns[:8])}{'...' if len(columns) > 8 else ''}")
            print()
            
        except Exception as e:
            dataset_summary.append({
                'Dataset': name,
                'Status': '⚠️ ERROR',
                'Rows': str(e)[:50],
                'Columns': '-',
                'Size (MB)': '-'
            })
            print(f"⚠️  {name}: Error - {str(e)[:50]}")
            print()
    else:
        dataset_summary.append({
            'Dataset': name,
            'Status': '❌',
            'Rows': 'Not found',
            'Columns': '-',
            'Size (MB)': '-'
        })
        print(f"❌ {name}: NOT FOUND")
        print()

# Display summary table
summary_df = pd.DataFrame(dataset_summary)
display(summary_df)

## 4. 🔍 Detailed Column Analysis

In [ ]:
print("=" * 80)
print("DETAILED COLUMN STRUCTURE")
print("=" * 80)
print()

# Crisis data
if Path('standardized_data/crisis_combined.csv').exists():
    print("📊 CRISIS COMBINED")
    print("-" * 80)
    crisis_df = pd.read_csv('standardized_data/crisis_combined.csv', nrows=3)
    print(f"Columns: {list(crisis_df.columns)}")
    print(f"\nData types:\n{crisis_df.dtypes}")
    print(f"\nFirst row sample:")
    display(crisis_df.head(1).T)
    print()

# Non-crisis data  
if Path('standardized_data/non_crisis_combined.csv').exists():
    print("\n📊 NON-CRISIS COMBINED")
    print("-" * 80)
    non_crisis_df = pd.read_csv('standardized_data/non_crisis_combined.csv', nrows=3)
    print(f"Columns: {list(non_crisis_df.columns)}")
    print(f"\nData types:\n{non_crisis_df.dtypes}")
    print(f"\nFirst row sample:")
    display(non_crisis_df.head(1).T)
    print()

# GoEmotions
if Path('goemotion_data/goemotions.csv').exists():
    print("\n📊 GOEMOTIONS (27 EMOTIONS)")
    print("-" * 80)
    goemo_df = pd.read_csv('goemotion_data/goemotions.csv', nrows=3)
    print(f"Columns: {list(goemo_df.columns)}")
    print(f"\nData types:\n{goemo_df.dtypes}")
    print(f"\nFirst row sample:")
    display(goemo_df.head(1).T)
else:
    print("\n❌ GoEmotions NOT FOUND - Need to download from Google Drive!")

## 5. ⚠️ Issues & Missing Data

In [ ]:
print("=" * 80)
print("ISSUES & MISSING COMPONENTS")
print("=" * 80)
print()

issues = []

# Check for GoEmotions
if not Path('goemotion_data/goemotions.csv').exists():
    issues.append("❌ GoEmotions data missing - CRITICAL for emotion mapping")

# Check for emotion columns in crisis/non-crisis data
if Path('standardized_data/crisis_combined.csv').exists():
    crisis_df = pd.read_csv('standardized_data/crisis_combined.csv', nrows=1)
    emotion_cols = [col for col in crisis_df.columns if 'emotion' in col.lower()]
    if not emotion_cols:
        issues.append("⚠️  No emotion columns in crisis_combined.csv - needs to be added")

if Path('standardized_data/non_crisis_combined.csv').exists():
    non_crisis_df = pd.read_csv('standardized_data/non_crisis_combined.csv', nrows=1)
    emotion_cols = [col for col in non_crisis_df.columns if 'emotion' in col.lower()]
    if not emotion_cols:
        issues.append("⚠️  No emotion columns in non_crisis_combined.csv - needs to be added")

# Check for master training data
if not Path('master_training_data/master_training_data.csv').exists():
    issues.append("⚠️  Master training file missing - may need to be regenerated")

if issues:
    for issue in issues:
        print(issue)
else:
    print("✅ No major issues detected!")

print()
print("=" * 80)
print("RECOMMENDED ACTIONS")
print("=" * 80)
print("""
1. Download goemotion_data/ from Google Drive (if missing)
2. Define 13 target emotions for crisis detection
3. Create 27→13 emotion mapping using LLM
4. Add emotion_label column to standardized datasets
5. Refactor code to industry standards
6. Convert Python scripts to notebooks
""")

## 6. 📈 Quick Statistics

In [ ]:
print("=" * 80)
print("QUICK STATISTICS")
print("=" * 80)
print()

stats = {}

# Crisis data stats
if Path('standardized_data/crisis_combined.csv').exists():
    crisis_df = pd.read_csv('standardized_data/crisis_combined.csv')
    print("📊 Crisis Data:")
    print(f"   Total tweets: {len(crisis_df):,}")
    if 'event_type' in crisis_df.columns:
        print(f"   Unique event types: {crisis_df['event_type'].nunique()}")
        print(f"   Top 5 event types:")
        for event, count in crisis_df['event_type'].value_counts().head(5).items():
            print(f"      {event}: {count:,}")
    print()

# Non-crisis data stats
if Path('standardized_data/non_crisis_combined.csv').exists():
    non_crisis_df = pd.read_csv('standardized_data/non_crisis_combined.csv')
    print("📊 Non-Crisis Data:")
    print(f"   Total tweets: {len(non_crisis_df):,}")
    if 'event_name' in non_crisis_df.columns:
        print(f"   Unique events: {non_crisis_df['event_name'].nunique()}")
        print(f"   Top 5 events:")
        for event, count in non_crisis_df['event_name'].value_counts().head(5).items():
            print(f"      {event}: {count:,}")
    print()

# Combined totals
if 'crisis_df' in dir() and 'non_crisis_df' in dir():
    total_tweets = len(crisis_df) + len(non_crisis_df)
    print(f"📊 TOTAL DATASET: {total_tweets:,} tweets")
    print(f"   Crisis: {len(crisis_df):,} ({len(crisis_df)/total_tweets*100:.1f}%)")
    print(f"   Non-crisis: {len(non_crisis_df):,} ({len(non_crisis_df)/total_tweets*100:.1f}%)")